In [5]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [6]:
from glob import glob
from tqdm import tqdm

txts = glob('../youtube/malay/*istana/*istana*/*.txt')
txts.extend(glob('../youtube/malay2/*turki/*turki*/*.txt'))
txts.extend(glob('../youtube/malay2/*turki-short/*turki*/*.txt'))
txts.extend(glob('../youtube/malay/*istana-short/*istana*/*.txt'))
len(txts)

38267

In [7]:
import malaya

In [8]:
model = malaya.true_case.transformer(model = 'small', quantized = True)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
import numpy as np

def compute_sparse_correlation_matrix(A):
    scaler = StandardScaler(with_mean = False)
    scaled_A = scaler.fit_transform(A)
    corr_matrix = (1 / scaled_A.shape[0]) * (scaled_A.T @ scaled_A)
    return corr_matrix

def filter_splitted(s, t, threshold = 0.001):
    bow = CountVectorizer(token_pattern = '[A-Za-z0-9]+').fit([s])
    s_bow = bow.transform([s] + t)
    score = np.array(compute_sparse_correlation_matrix(s_bow.T).todense())[0,1:]
    r = [t[no] for no, s in enumerate(score) if s >= threshold]
    return ' '.join(r)

In [17]:
results = []
batch_size = 1
for i in tqdm(range(0, len(txts), batch_size)):
    index = min(i + batch_size, len(txts))
    batch = txts[i: index]
    r = []
    for t in batch:
        with open(t) as fopen:
            r.append(fopen.read())
    
    c = model.true_case(r, beam_search = False)

    for k in range(len(batch)):
        s = r[k]
        t = malaya.text.function.split_into_sentences(c[k])
        l = len(malaya.text.function.split_into_sentences(s))
    
    results.extend(list(zip(batch, r, c)))

100%|██████████| 38267/38267 [1:50:43<00:00,  5.76it/s]  


In [18]:
results

[('../youtube/malay/dari-pasentran-ke-istana/output-text-dari-pasentran-ke-istana/perubahan-rejim-dan-pergelutan-demi-kelangsungan-12.mp3-26.wav.txt',
  'the Jakarta post menyebut pada 1 September',
  'The Jakarta Post menyebut pada 1 September.'),
 ('../youtube/malay/dari-pasentran-ke-istana/output-text-dari-pasentran-ke-istana/kaherah-baghdad-dan-eropah-3.mp3-134.wav.txt',
  'Mungkin ada juga manfaat yang boleh diraih dengan menempuh pengajian peringkat Sarjana Muda di al-azhar seperti terbukti pada banyak mahasiswa Indonesia yang lain',
  'Mungkin ada juga manfaat yang boleh diraih dengan menempuh pengajian peringkat Sarjana Muda di al-Azhar seperti terbukti pada banyak mahasiswa Indonesia yang lain.'),
 ('../youtube/malay/dari-pasentran-ke-istana/output-text-dari-pasentran-ke-istana/islam-politik-dan-pilihanraya-10.mp3-157.wav.txt',
  'banyak kelompok dalam bersaing untuk mendapatkan naungan Gusdur pada mulanya Gusdur meluahkan kebimbangan melihat kumpulan-kumpulan dalam anu mahu m

In [19]:
import json
from glob import glob
from tqdm import tqdm

with open('true-case-male.json') as fopen:
    txts = json.load(fopen)
    
len(txts), txts[0]

In [ ]:
import json

with open('')